## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('/Users/seanvillanoy/Documents/Github/Challenge-6/Weather_Database/WeatherPy_database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.97,102.50,15.82,96,100,21.99,overcast clouds
1,1,Yulara,AU,-25.24,130.99,84.20,26,98,16.11,overcast clouds
2,2,Punta Arenas,CL,-53.15,-70.92,55.40,54,75,25.28,broken clouds
3,3,Atuona,PF,-9.80,-139.03,79.14,78,6,20.13,clear sky
4,4,Lima,PE,-12.04,-77.03,71.60,68,20,10.29,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Yulara,AU,-25.24,130.99,84.20,26,98,16.11,overcast clouds
3,3,Atuona,PF,-9.80,-139.03,79.14,78,6,20.13,clear sky
9,9,Lubao,PH,14.92,120.55,77.00,83,75,4.70,broken clouds
13,13,Makakilo City,US,21.35,-158.09,78.80,61,20,9.17,few clouds
18,18,Butaritari,KI,3.07,172.79,81.90,77,37,21.41,scattered clouds
22,22,Rikitea,PF,-23.12,-134.97,76.32,81,92,11.90,light rain
29,29,Empalme,MX,27.97,-110.82,82.99,16,5,14.99,clear sky
34,34,Port-Gentil,GA,-0.72,8.78,80.60,88,40,8.05,scattered clouds
38,38,Kapaa,US,22.08,-159.32,78.80,69,20,13.87,few clouds
39,39,Carutapera,BR,-1.20,-46.02,79.81,88,74,7.63,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                189
City                   189
Country                187
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                187
City                   187
Country                187
Lat                    187
Lng                    187
Max Temp               187
Humidity               187
Cloudiness             187
Wind Speed             187
Current Description    187
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yulara,AU,84.20,overcast clouds,-25.24,130.99,
3,Atuona,PF,79.14,clear sky,-9.80,-139.03,
9,Lubao,PH,77.00,broken clouds,14.92,120.55,
13,Makakilo City,US,78.80,few clouds,21.35,-158.09,
18,Butaritari,KI,81.90,scattered clouds,3.07,172.79,
22,Rikitea,PF,76.32,light rain,-23.12,-134.97,
29,Empalme,MX,82.99,clear sky,27.97,-110.82,
34,Port-Gentil,GA,80.60,scattered clouds,-0.72,8.78,
38,Kapaa,US,78.80,few clouds,22.08,-159.32,
39,Carutapera,BR,79.81,broken clouds,-1.20,-46.02,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]   
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Determine if there are any empty rows.
for index, row in hotel_df.iterrows():
    # change any empty string under 'Hotel Name' to a null value
    if hotel_df.loc[index,"Hotel Name"]=='':
        hotel_df.loc[index,"Hotel Name"]= np.nan
hotel_df.count()

City                   187
Country                187
Max Temp               187
Current Description    187
Lat                    187
Lng                    187
Hotel Name             175
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   175
Country                175
Max Temp               175
Current Description    175
Lat                    175
Lng                    175
Hotel Name             175
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather </dt><dd>{Current Description}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))